In [ ]:
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from matplotlib import style
import tkinter as tk
from tkinter import ttk

In [ ]:
class channel:
    """The channel class represents a MIPS output voltage which
    applies the voltage on the SLIM instrument."""
    
    def __init__(self, description, voltage):
        self.description = description
        self.voltage = voltage
    
    def set_voltage(self, new_voltage):
        self.voltage = new_voltage

class region:
    """The region class represents the space between two voltage inputs. 
    Over a defined length, the two voltage inputs create the separation field."""
    def __init__(self, description, length, field_strength, rungs=None):
        self.description = description
        self.length = length
        self.field_strength = field_strength
        self.rungs=rungs
        
    def set_length(self, new_length):
        self.length = new_length
        
    def set_field_strength(self, new_field_strength):
        self.field_strength = new_field_strength
        
    def voltage_drop(self):
        voltage_drop = self.field_strength * self.length
        return voltage_drop
    
    def voltage_drop_per_rung(self):
        if self.rungs == None:
            raise ValueError('Number of rungs undefined.')
        else:
            voltage_drop_per_rung = self.voltage_drop() / self.rungs
            return voltage_drop_per_rung

In [ ]:
# Define the region and channels of the CIF
CIF = region('CIF', 10.87, 0)
CSSO = channel('CCSO', 0)
CIF_CL_to_CCSO_drop = 0
CIF_CL = channel('CIFCL', 0)
CIF_to_CIF_CL_drop = 0
CIF_dc_out = channel('CIF_dc_out', 0)
CIF_dc_in = channel('CIF_dc_in', 0)

# Define the region and channels of SLIM 2
SLIM2 = region('SLIM2', 38.1, 0, rungs=250)
SLIM2_dc_in = channel('SLIM2_dc_in', 0)
SLIM2_dc_out = channel('SLIM2_dc_out', 0)
M8_sg = channel('Module 8 switch guard', 0)
SLIM2_to_CIF_drop = 0
M8_sg_bias = 0

#Define channels in the interface region
M8_orth_out = channel('M8_orth_out', 0)
M8_orth_in = channel('M8_orth_in', 0)
M8_orth = region('M8_orth', 3.5052, 0)
SLIM2_8M_interface = 0
M8_orth_to_SLIM2 = 0

#Define channels in the interface region
M7_orth_out = channel('M7_orth_out', 0)
M7_orth_in = channel('M7_orth_in', 0)
M7_orth = region('M7_orth', 3.5052, 0)
SLIM1_7M_interface = 0

# #Define drops in the region
M7_to_M8_drop = 0

#Define channels and regions
SLIM1_dc_out = channel('SLIM1_dc_out', 0)
SLIM1_dc_in = channel('SLIM1_dc_in', 0)
M7_sg = channel('Module 7 switch guard', 0)
SLIM1 = region('SLIM1', 45.72, 0, rungs=300)

#Define interface and voltage drops
SLIM1_to_7M_orth_drop = 0
M7_sg_bias = 0

In [ ]:
def set_voltages():
    #Set CIF voltages
    CIF_CL.set_voltage(CSSO.voltage + CIF_CL_to_CCSO_drop)
    CIF_dc_out.set_voltage(CIF_CL.voltage + CIF_to_CIF_CL_drop)
    CIF_dc_in.set_voltage(CIF_dc_out.voltage + (CIF.voltage_drop()))

    #Set SLIM2 voltages
    SLIM2_dc_out.set_voltage(CIF_dc_in.voltage + SLIM2_to_CIF_drop)
    SLIM2_dc_in.set_voltage(SLIM2_dc_out.voltage + (SLIM2.field_strength * SLIM2.length))
    global SLIM2_8M_interface
    SLIM2_8M_interface = (SLIM2_dc_in.voltage - (SLIM2.voltage_drop() * (25.5/250)))
    M8_sg.set_voltage(SLIM2_8M_interface + M8_sg_bias)

    #Set 8M voltages
    M8_orth_out.set_voltage(SLIM2_8M_interface + M8_orth_to_SLIM2)
    M8_orth_in.set_voltage(M8_orth_out.voltage + M8_orth.voltage_drop())

    #Set 7M voltages
    M7_orth_out.set_voltage(M8_orth_in.voltage + M7_to_M8_drop)
    M7_orth_in.set_voltage(M7_orth_out.voltage + M7_orth.voltage_drop())
    global SLIM1_7M_interface
    SLIM1_7M_interface = M7_orth_in.voltage + SLIM1_to_7M_orth_drop
    M7_sg.set_voltage(SLIM1_7M_interface + M8_sg_bias)

    #Set voltages
    SLIM1_dc_out.set_voltage(SLIM1_7M_interface - SLIM1.voltage_drop()*(25.5/300))
    SLIM1_dc_in.set_voltage(SLIM1_dc_out.voltage + SLIM1.voltage_drop())

In [ ]:
LARGE_FONT = ("Verdana", 10)
style.use("ggplot")

# def animate(i):
#     pullData = open("sampleData.txt","r").read()
#     dataList = pullData.split('\n')
#     xList = []
#     yList = []
#     for eachLine in dataList:
#         if len(eachLine)>1:
#             x, y = eachLine.split(',')
#             xList.append(int(x))
#             yList.append(int(y))
#     a0.clear()
#     a0.plot(xList, yList)
    
def animate(i):
    CIF_loc = [0,-10.87]
    CIF_vol = [CIF_dc_out.voltage, CIF_dc_in.voltage]
    SLIM2_loc = [-10.87, -48.97]
    SLIM2_vol = [SLIM2_dc_out.voltage, SLIM2_dc_in.voltage]
    M8_orth_loc = [-45.0838,-48.589]
    M8_orth_vol = [M8_orth_out.voltage, M8_orth_in.voltage]
    M7_orth_loc = [-48.589, -52.0942]
    M7_orth_vol = [M7_orth_out.voltage, M7_orth_in.voltage]
    SLIM1_loc = [-48.208,-93.928]
    SLIM1_vol = [SLIM1_dc_out.voltage, SLIM1_dc_in.voltage]
    a0.clear()
    a1.clear()
    a0.plot(CIF_loc, CIF_vol,label='CIF')
    a0.plot(SLIM2_loc, SLIM2_vol, label='SLIM2')
    a0.plot(M8_orth_loc, M8_orth_vol, label='8M Orth')
    a0.plot(M7_orth_loc, M7_orth_vol, label='7M Orth')
    a0.plot(SLIM1_loc, SLIM1_vol, label='SLIM1')
    a0.legend()
    #a0.title('Drift Region Voltages')
    a1.plot(M8_orth_loc, M8_orth_vol, label='8M Orth')
    a1.plot(M7_orth_loc, M7_orth_vol, label='7M Orth')
    a1.scatter(-45.0838, SLIM2_8M_interface)
    a1.scatter(-52.0942, SLIM1_7M_interface)
    #a1.title('Orthogonal Region Voltages')
    

f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios':[3,2]})
f.set_figheight(5)
f.set_figwidth(10)

class SLIMVoltagesApplication(tk.Tk):
    
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        #tk.Tk.iconbitmap(self, default="insertthislater")
        tk.Tk.wm_title(self, "SLIM Voltages")
        
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        
        self.frames = {}
              
        for F in (VoltageControl,SignalTable):
        
            frame = F(container, self)

            self.frames[F] = frame
            
            frame.grid(row=0, column=0, sticky="nsew")
    
        self.show_frame(VoltageControl)
    
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

        

class VoltageControl(tk.Frame):
    
    def __init__(self, parent, controller): 
        tk.Frame.__init__(self, parent)
        button_dead = ttk.Button(self, text="Voltage Control")
        button_dead.grid(column=0, row=0)
        button = ttk.Button(self, text="Signal Tables", 
                            command=lambda: controller.show_frame(SignalTable))
        button.grid(column=1, row=0)
        
        canvas = FigureCanvasTkAgg(f, self)
        canvas.draw()
        canvas.get_tk_widget().grid(column=5, rowspan=20)
        
        #CIF Region
        ccso_lbl = tk.Label(self, text="Collision Cell Static Offset", font=LARGE_FONT)
        ccso_lbl.grid(column=0, row=1)
        ccso_entry = tk.Entry(self, width=5)
        ccso_entry.grid(column=1, row=1)

        CIF_to_CSSO_drop_lbl = tk.Label(self, text="CIF CL to CSSO drop", font=LARGE_FONT)
        CIF_to_CSSO_drop_lbl.grid(column=0, row=2)
        CIF_to_CSSO_drop_entry = tk.Entry(self, width=5)
        CIF_to_CSSO_drop_entry.grid(column=1, row=2)

        CIF_to_CIF_CL_drop_lbl = tk.Label(self, text="CIF to CIF CL drop", font=LARGE_FONT)
        CIF_to_CIF_CL_drop_lbl.grid(column=0, row=3)
        CIF_to_CIF_CL_drop_entry = tk.Entry(self, width=5)
        CIF_to_CIF_CL_drop_entry.grid(column=1, row=3)

        CIF_fs_lbl = tk.Label(self, text="CIF field strength", font=LARGE_FONT)
        CIF_fs_lbl.grid(column=0, row=4)
        CIF_fs_entry = tk.Entry(self, width=5)
        CIF_fs_entry.grid(column=1, row=4)

        #SLIM2 Region
        SLIM2_to_CIF_drop_lbl = tk.Label(self, text="SLIM2 to CIF drop", font=LARGE_FONT)
        SLIM2_to_CIF_drop_lbl.grid(column=0, row=5)
        SLIM2_to_CIF_drop_entry = tk.Entry(self, width=5)
        SLIM2_to_CIF_drop_entry.grid(column=1,row=5)

        SLIM2_fs_lbl = tk.Label(self, text="SLIM2 Field Strength", font=LARGE_FONT)
        SLIM2_fs_lbl.grid(column=0, row=6)
        SLIM2_fs_entry = tk.Entry(self, width=5)
        SLIM2_fs_entry.grid(column=1, row=6)

        #Orthogonal Region
        M8_sg_bias_lbl = tk.Label(self, text="8M Switch Guard Bias", font=LARGE_FONT)
        M8_sg_bias_lbl.grid(column=0, row=7)
        M8_sg_bias_entry = tk.Entry(self, width=5)
        M8_sg_bias_entry.grid(column=1, row=7)

        M8_orth_fs_lbl = tk.Label(self, text="8M Orth Field Strength", font=LARGE_FONT)
        M8_orth_fs_lbl.grid(column=0, row=8)
        M8_orth_fs_entry = tk.Entry(self, width=5)
        M8_orth_fs_entry.grid(column=1, row=8)

        M8_orth_to_SLIM2_lbl = tk.Label(self, text="8M to SLIM2 Drop", font=LARGE_FONT)
        M8_orth_to_SLIM2_lbl.grid(column=0, row=9)
        M8_orth_to_SLIM2_entry = tk.Entry(self, width=5)
        M8_orth_to_SLIM2_entry.grid(column=1, row=9)

        M7_to_M8_drop_lbl = tk.Label(self, text="7M to 8M voltage drop", font=LARGE_FONT)
        M7_to_M8_drop_lbl.grid(column=0, row=10)
        M7_to_M8_drop_entry = tk.Entry(self, width=5)
        M7_to_M8_drop_entry.grid(column=1, row=10)

        M7_orth_fs_lbl = tk.Label(self, text="7M Orth Field Strength", font=LARGE_FONT)
        M7_orth_fs_lbl.grid(column=0, row=11)
        M7_orth_fs_entry = tk.Entry(self, width=5)
        M7_orth_fs_entry.grid(column=1, row=11)

        #SLIM 1
        SLIM1_to_7M_orth_drop_lbl = tk.Label(self, text="SLIM1 to 7M voltage drop", font=LARGE_FONT)
        SLIM1_to_7M_orth_drop_lbl.grid(column=0, row=12)
        SLIM1_to_7M_orth_drop_entry = tk.Entry(self, width=5)
        SLIM1_to_7M_orth_drop_entry.grid(column=1, row=12)

        M7_sg_bias_lbl = tk.Label(self, text="7M SG Bias", font=LARGE_FONT)
        M7_sg_bias_lbl.grid(column=0, row=13)
        M7_sg_bias_entry = tk.Entry(self, width=5)
        M7_sg_bias_entry.grid(column=1, row=13)

        SLIM1_fs_lbl = tk.Label(self, text="SLIM1 Field Strength", font=LARGE_FONT)
        SLIM1_fs_lbl.grid(column=0, row=14)
        SLIM1_fs_entry = tk.Entry(self, width=5)
        SLIM1_fs_entry.grid(column=1, row=14)
        
        def update_voltage_settings():
            try:
                CSSO.set_voltage(float(ccso_entry.get()))
                global CIF_CL_to_CCSO_drop
                CIF_CL_to_CCSO_drop = float(CIF_to_CSSO_drop_entry.get())
                global CIF_to_CIF_CL_drop
                CIF_to_CIF_CL_drop = float(CIF_to_CIF_CL_drop_entry.get())
                CIF.set_field_strength(float(CIF_fs_entry.get()))
                global SLIM2_to_CIF_drop
                SLIM2_to_CIF_drop = float(SLIM2_to_CIF_drop_entry.get())
                SLIM2.set_field_strength(float(SLIM2_fs_entry.get()))
                global M8_sg_bias
                M8_sg_bias = float(M8_sg_bias_entry.get())
                M8_orth.set_field_strength(float(M8_orth_fs_entry.get()))
                global M8_orth_to_SLIM2
                M8_orth_to_SLIM2 = float(M8_orth_to_SLIM2_entry.get())
                global M7_to_M8_drop
                M7_to_M8_drop = float(M7_to_M8_drop_entry.get())
                M7_orth.set_field_strength(float(M7_orth_fs_entry.get()))
                global SLIM1_to_7M_orth_drop
                SLIM1_to_7M_orth_drop = float(SLIM1_to_7M_orth_drop_entry.get())
                SLIM1.set_field_strength(float(SLIM1_fs_entry.get()))
                global M7_sg_bias
                M7_sg_bias = float(M7_sg_bias_entry.get())
                error_label.configure(text="")
            except ValueError:
                error_label.configure(text="Voltage entries could not be converted to numbers.")
        
        update_voltages_btn = tk.Button(self, text="Update Voltage Inputs", command=update_voltage_settings)
        update_voltages_btn.grid(column=0, row=15)

        error_label = tk.Label(self, text=None, font=LARGE_FONT)
        error_label.grid(column=0, row=16)
        
        set_voltages_btn = tk.Button(self, text="Set Voltages", command=set_voltages)
        set_voltages_btn.grid(column=0, row=17)
        

class SignalTable(tk.Frame):
    
    def __init__(self, parent, controller): 
        tk.Frame.__init__(self, parent)
        button = ttk.Button(self, text="Voltage Control", 
                            command=lambda: controller.show_frame(VoltageControl))
        button.grid(row=0, column=0)
        button_dead = ttk.Button(self, text="Signal Tables")
        button_dead.grid(row=0,column=1)


app = SLIMVoltagesApplication()
ani = animation.FuncAnimation(f, animate, interval=1000)
app.mainloop()

In [ ]:
CSSO.voltage

In [ ]:
CIF_CL_to_CCSO_drop

In [ ]:
CIF_to_CIF_CL_drop

In [ ]:
CIF.field_strength

In [ ]:
SLIM2_to_CIF_drop

In [ ]:
SLIM2.field_strength

In [ ]:
M8_sg_bias

In [ ]:
M8_orth.field_strength

In [ ]:
M8_orth_to_SLIM2

In [ ]:
M7_to_M8_drop

In [ ]:
M7_orth.field_strength

In [ ]:
SLIM1_to_7M_orth_drop

In [ ]:
SLIM1.field_strength

In [ ]:
M7_sg_bias

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios':[3,2]})
f.set_figheight(5)
f.set_figwidth(10)
#full SLIM plot
a0.set_title('Drift Region')
a0.plot([0,10.87,10.87],[CIF_dc_in.voltage,CIF_dc_out.voltage,CIF_CL.voltage])
a0.plot([-38.1,0,], [SLIM2_dc_in.voltage, SLIM2_dc_out.voltage])
a0.plot([-34.2138, -37.719], [M8_orth_out.voltage, M8_orth_in.voltage])
a0.plot([-37.719, -41.2242], [M7_orth_out.voltage, M7_orth_in.voltage])
a0.plot([-37.338, -83.058], [SLIM1_dc_out.voltage, SLIM1_dc_in.voltage])

#tee interface plot
a1.set_title('Tee Interface')
a1.scatter([-34.2138],[M8_sg.voltage])
a1.scatter([-34.2138],[SLIM2_8M_interface])
a1.plot([-34.2138, -37.719], [M8_orth_out.voltage, M8_orth_in.voltage])
a1.plot([-37.719, -41.2242], [M7_orth_out.voltage, M7_orth_in.voltage])
a1.scatter([-41.2242],[M7_sg.voltage])
a1.scatter([-41.2242],[SLIM1_7M_interface])
#a1.scatter(SLIM1_7M_interface_rung_location, SLIM1_7M_interface_new)

In [ ]:
f.show()